In [1]:
# Import the required libraries
import pandas as pd
import numpy as np

### Loading the dataset

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# Creating the feature matrix
X = df.drop('label', axis = 1)

In [4]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [5]:
# Get the dependent faeatures
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [6]:
# Shape of the data
df.shape

(20800, 5)

In [7]:
# Removing the NaN values
df = df.dropna()

In [8]:
df.shape

(18285, 5)

In [10]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
# Vocabulary size
vocab_size = 5000

### One-Hot Representation

In [12]:
messages = df.copy()

In [13]:
# Resetting the indices after the removal of NaN values
messages.reset_index(inplace = True)

In [18]:
messages['title'][5]

'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart'

#### Data pre-processing

In [16]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [20]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_rep = [one_hot(words,vocab_size) for words in corpus]
onehot_rep

[[3253, 4237, 2046, 1124, 1806, 3373, 1783, 2821, 4594, 4036],
 [4988, 3378, 944, 2918, 4323, 312, 1890],
 [613, 222, 683, 2187],
 [4315, 1248, 4619, 3395, 1125, 1109],
 [4870, 4323, 3929, 2243, 4354, 1087, 4323, 1578, 1122, 1808],
 [3775,
  3693,
  2995,
  4447,
  1472,
  1993,
  1880,
  4159,
  450,
  2271,
  1016,
  2822,
  4518,
  1680,
  1890],
 [675, 85, 2634, 209, 2260, 4648, 3497, 1156, 3906, 1404, 1272],
 [2258, 735, 998, 4799, 3276, 2241, 1993, 3261, 3906, 1404, 1272],
 [365, 836, 1177, 2100, 2602, 2405, 4500, 2731, 1993, 3329],
 [2015, 4942, 4343, 264, 746, 2119, 774, 2309],
 [117, 2998, 1258, 758, 3740, 4951, 3838, 1445, 4685, 1863, 1448],
 [3395, 3826, 1806, 2405, 1993, 3276],
 [2559, 4166, 3985, 838, 1713, 1887, 2515, 402, 2423],
 [3082, 702, 1948, 4323, 4963, 4321, 1251, 3906, 1404, 1272],
 [712, 3486, 1167, 1340, 1793, 3906, 1404, 1272],
 [2193, 2017, 690, 2955, 80, 3485, 1910, 1303, 1867, 693],
 [2196, 4432, 3378],
 [569, 3327, 3013, 591, 1993, 2562, 3099, 1890],
 [494

### Embedding Representation

In [24]:
sent_len = 20
embedded = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_len)
print(embedded)

[[   0    0    0 ... 2821 4594 4036]
 [   0    0    0 ... 4323  312 1890]
 [   0    0    0 ...  222  683 2187]
 ...
 [   0    0    0 ... 3906 1404 1272]
 [   0    0    0 ... 2097 2699 1971]
 [   0    0    0 ... 3221 1678 4980]]


In [25]:
len(embedded)

18285

In [26]:
embedded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3253,
       4237, 2046, 1124, 1806, 3373, 1783, 2821, 4594, 4036], dtype=int32)

### Creating the model

In [27]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
y = messages['label']

In [35]:
import numpy as np
X_final = np.array(embedded)
y_final = np.array(y)

In [37]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [40]:
# Train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
192/192 [==============================] - 3s 16ms/step - loss: 0.0097 - accuracy: 0.9970 - val_loss: 0.5270 - val_accuracy: 0.9153
Epoch 2/20
192/192 [==============================] - 3s 15ms/step - loss: 0.0038 - accuracy: 0.9994 - val_loss: 0.6288 - val_accuracy: 0.9123
Epoch 3/20
192/192 [==============================] - 3s 17ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.6084 - val_accuracy: 0.9122
Epoch 4/20
192/192 [==============================] - 3s 18ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 0.6357 - val_accuracy: 0.9120
Epoch 5/20
192/192 [==============================] - 3s 18ms/step - loss: 0.0068 - accuracy: 0.9984 - val_loss: 0.6141 - val_accuracy: 0.9125
Epoch 6/20
192/192 [==============================] - 3s 17ms/step - loss: 0.0057 - accuracy: 0.9984 - val_loss: 0.5596 - val_accuracy: 0.9099
Epoch 7/20
192/192 [==============================] - 3s 16ms/step - loss: 0.0040 - accuracy: 0.9984 - val_loss: 0.6481 - val_accuracy: 0.9118

### Performance Metrics

In [44]:
y_pred = (model.predict_classes(X_test))

In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)

0.9121789560894781